## OPTIMISED MODEL FITTING 



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
import warnings
import os
import time
from pathlib import Path
from multiprocessing import Pool, cpu_count
from scipy.optimize import differential_evolution
from joblib import Parallel, delayed


#warnings.filterwarnings("ignore", message="Glyph.*missing from font")
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 4)


### Step 1: Task and environment generation

In [5]:
@dataclass
class VolatileBanditTask:
    """Container that stores a whole task instance for re-use across agents."""
    stimuli: np.ndarray  # here it is always zero but kept for completeness
    latent_states: np.ndarray  # z_t, the true task-set index
    switch_prob: np.ndarray  # τ_t
    traps: np.ndarray  # Bernoulli noise on rewards
    correct_actions: np.ndarray  # action that yields positive feedback if there is no trap
    beta: float  # feedback reliability parameter η in the paper



### Step 2: Particle filtering utilities - the sequential Monte Carlo (SMC) algorithm



In [6]:
# ============================================================
# GLOBAL CONSTANTS (required by agent classes)
# ============================================================
K = 2  # Number of task-sets (latent states) in closed 2-armed bandit

# TASK_MAPPING[stimulus] returns array of correct actions for each task-set
# For closed environment: stimulus 0, task-set 0 -> action 0, task-set 1 -> action 1
TASK_MAPPING = np.array([[0, 1]])  # Shape: (n_stimuli, K) = (1, 2)

def stratified_resample(weights: np.ndarray, rng: np.random.Generator) -> np.ndarray:
    """Stratified resampling as used in the original C++ code."""
    n = len(weights)
    cumulative = np.cumsum(weights)
    positions = (rng.random(n) + np.arange(n)) / n
    return np.searchsorted(cumulative, positions)


def sample_inv_gamma(shape: float, scale: float, size: int, rng: np.random.Generator) -> np.ndarray:
    """Inverse-gamma sampler matching utils/useful_functions.py."""
    return scale / rng.gamma(shape, size=size)


def logsumexp(log_w: np.ndarray) -> float:
    b = np.max(log_w)
    return b + np.log(np.sum(np.exp(log_w - b)))


def normalise_log_weights(log_w: np.ndarray) -> np.ndarray:
    """Convert log-weights to normalized weights without numerical trouble."""
    log_w = log_w - logsumexp(log_w)
    return np.exp(log_w)


def compute_positive_states(mapping: np.ndarray, stimulus: int, action: int, reward: int) -> np.ndarray:
    """Replicates isEqual_and_adapted_logical_xor from the C++ helpers."""
    equals = mapping[stimulus] == action
    if reward:  # reward==1 means the action was *reported* as correct
        return equals.astype(float)
    return (~equals).astype(float)


def sample_new_state_excluding(current: int, gamma: np.ndarray, rng: np.random.Generator) -> int:
    probs = gamma.copy()
    probs[current] = 0.0
    total = probs.sum()
    if total == 0:
        probs[:] = 1.0 / len(probs)
    else:
        probs /= total
    return rng.choice(len(probs), p=probs)



### Step 3a: Forward varying-volatility agent


In [7]:
class ForwardVaryingVolatilityAgent:
    """Forward (online) SMC approximation of the exact varying-volatility model.
    
    Parameters (Paper vs Code mapping):
    ------------------------------------
    LATENT PARAMETERS (inferred by SMC):
    - beta_samples (internal): η/2 sampled from Beta(1,1), transformed to η = beta/2 + 0.5
      This is the FEEDBACK NOISE parameter (η > 0.5 in paper)
    - nu_samples (internal): ν sampled from Inv-Gamma(3, 0.001)
      This is the VOLATILITY VARIANCE parameter
    - gamma_samples (internal): γ sampled from Dirichlet(1,...,1)
      These are the task-set COMBINATION PROBABILITIES
    
    LATENT STATES (tracked by particle filter):
    - state_particles: z_t, the current task-set hypothesis
    - tau_particles: τ_t, the current volatility (switch probability)
    
    FREE PARAMETERS (for model fitting):
    - beta_softmax: β, the softmax inverse temperature
      NOTE: This is DIFFERENT from beta_samples (feedback noise η)!
      When None, uses argmax policy (deterministic).
    - epsilon_softmax: Lapse rate for random exploration
    """

    def __init__(
        self,
        num_theta: int = 200, # simulation uses 1000, fitting only 200
        num_state_particles: int = 200, # simulation uses 1000-2000, fitting o
        tau_default: float = 0.03,
        tau_bounds: Tuple[float, float] = (0.0, 1), # c++ code uses 0-1 even though this would be an extremely high volatile environment, but for Findling constistency - kept the same. 
        beta_prior: Tuple[float, float] = (1.0, 1.0),
        nu_prior: Tuple[float, float] = (3.0, 1e-3),
        gamma_prior: float = 1.0,
        ess_threshold: float = 0.5,
        beta_softmax: Optional[float] = 1.0,  # None in simulation for argmax policy
        epsilon_softmax: float = 0.0,
        seed: Optional[int] = None,
    ):
        self.num_theta = num_theta
        self.num_state_particles = num_state_particles
        self.tau_default = tau_default
        self.tau_bounds = tau_bounds
        self.beta_prior = beta_prior
        self.nu_prior = nu_prior
        self.gamma_prior = gamma_prior
        self.ess_threshold = ess_threshold
        self.beta_softmax = beta_softmax
        self.epsilon_softmax = epsilon_softmax
        self.rng = np.random.default_rng(seed)

        self._init_particles()

    def _init_particles(self):
        self.beta_samples = self.rng.beta(
            self.beta_prior[0], self.beta_prior[1], size=self.num_theta
        )
        self.nu_samples = sample_inv_gamma(
            self.nu_prior[0], self.nu_prior[1], size=self.num_theta, rng=self.rng
        )
        self.gamma_samples = self.rng.dirichlet(
            np.ones(K) * self.gamma_prior, size=self.num_theta
        )
        self.state_particles = self.rng.integers(
            K, size=(self.num_theta, self.num_state_particles)
        )
        self.tau_particles = np.full(
            (self.num_theta, self.num_state_particles), self.tau_default
        )
        self.log_theta_weights = np.zeros(self.num_theta)
        self.theta_weights = np.ones(self.num_theta) / self.num_theta
        self.pending_observation = None
        self.current_action = None
        self.current_stimulus = None
        self.trial_index = 0

        self.history = {
            "vol_mean": [],
            "beta_mean": [],
            "nu_mean": [],
            "ts_prob": [],
        }

    # ------------------------------------------------------------------
    # Agent interaction API
    # ------------------------------------------------------------------
    def act(self, stimulus: int) -> Tuple[int, np.ndarray]:
        self._maybe_smc_update()
        action_probs = self._action_distribution(stimulus)
        action = self._sample_action(action_probs)
        self.current_action = action
        self.current_stimulus = stimulus
        return action, action_probs

    def observe(self, reward: int):
        self.pending_observation = (
            self.current_stimulus,
            self.current_action,
            reward,
        )
        self.trial_index += 1

    # ------------------------------------------------------------------
    # SMC internals
    # ------------------------------------------------------------------
    def _maybe_smc_update(self):
        if self.pending_observation is None:
            return

        stimulus, action, reward = self.pending_observation
        prev_states = self.state_particles.copy()
        prev_taus = self.tau_particles.copy()

        positive_states = compute_positive_states(TASK_MAPPING, stimulus, action, reward)
        # Transform beta from [0,1] to [0.5,1] as in original code: betaSamples/2 + 0.5
        # This ensures feedback noise η > 0.5 (reward is more likely when action is correct)
        beta_transformed = self.beta_samples / 2.0 + 0.5
        beta_matrix = beta_transformed[:, None]
        state_pos = positive_states[prev_states]
        likelihoods = state_pos * beta_matrix + (1 - state_pos) * (1 - beta_matrix)
        likelihoods = np.clip(likelihoods, 1e-9, None)

        sum_weights = likelihoods.sum(axis=1)
        self.log_theta_weights += np.log(sum_weights / self.num_state_particles)
        weights_norm = likelihoods / sum_weights[:, None]

        new_states = np.empty_like(prev_states)
        new_taus = np.empty_like(prev_taus)
        for theta_idx in range(self.num_theta):
            ancestor_idx = stratified_resample(weights_norm[theta_idx], self.rng)
            new_states[theta_idx] = prev_states[theta_idx, ancestor_idx]
            new_taus[theta_idx] = prev_taus[theta_idx, ancestor_idx]

        noise = self.rng.normal(
            0.0,
            np.sqrt(self.nu_samples)[:, None],
            size=new_taus.shape,
        )
        tau_candidates = np.clip(
            new_taus + noise, self.tau_bounds[0], self.tau_bounds[1]
        )
        self.tau_particles = tau_candidates

        switch_mask = self.rng.random(size=new_states.shape) < tau_candidates
        propagated_states = new_states.copy()
        for theta_idx in range(self.num_theta):
            gamma = self.gamma_samples[theta_idx]
            indices = np.where(switch_mask[theta_idx])[0]
            for idx in indices:
                propagated_states[theta_idx, idx] = sample_new_state_excluding(
                    propagated_states[theta_idx, idx], gamma, self.rng
                )
        self.state_particles = propagated_states

        self.pending_observation = None
        self.theta_weights = normalise_log_weights(self.log_theta_weights)
        self._maybe_rejuvenate()
        self._record_history()

    def _maybe_rejuvenate(self):
        ess = 1.0 / np.sum(self.theta_weights ** 2)
        if ess >= self.ess_threshold * self.num_theta:
            return

        beta_mu = np.sum(self.theta_weights * self.beta_samples)
        beta_var = np.sum(self.theta_weights * (self.beta_samples - beta_mu) ** 2)
        beta_var = max(beta_var, 1e-6)
        beta_alpha = max(((1 - beta_mu) / beta_var - 1 / beta_mu) * beta_mu ** 2, 1.0)
        beta_beta = max(beta_alpha * (1 / beta_mu - 1), 1.0)

        nu_mu = np.sum(self.theta_weights * self.nu_samples)
        nu_var = np.sum(self.theta_weights * (self.nu_samples - nu_mu) ** 2)
        nu_var = max(nu_var, 1e-6)
        nu_alpha = nu_mu ** 2 / nu_var + 2.0
        nu_beta = nu_mu * (nu_alpha - 1)

        dirichlet_means = np.sum(
            self.theta_weights[:, None] * self.gamma_samples, axis=0
        )
        dirichlet_vars = np.sum(
            self.theta_weights[:, None] * (self.gamma_samples ** 2), axis=0
        ) - dirichlet_means ** 2
        dirichlet_vars = np.clip(dirichlet_vars, 1e-6, None)
        dirichlet_precision = (
            np.sum(dirichlet_means - dirichlet_means ** 2) / np.sum(dirichlet_vars)
        ) - 1
        dirichlet_precision = max(dirichlet_precision, 1.0)
        dirichlet_params = np.maximum(
            dirichlet_means * dirichlet_precision, 1.0
        )

        self.beta_samples = self.rng.beta(
            beta_alpha, beta_beta, size=self.num_theta
        )
        self.nu_samples = sample_inv_gamma(
            nu_alpha, nu_beta, size=self.num_theta, rng=self.rng
        )
        self.gamma_samples = self.rng.dirichlet(
            dirichlet_params, size=self.num_theta
        )
        self.log_theta_weights[:] = 0.0
        self.theta_weights = np.ones(self.num_theta) / self.num_theta

    def _record_history(self):
        tau_means = self.tau_particles.mean(axis=1)
        vol_mean = np.sum(self.theta_weights * tau_means)
        beta_mean = np.sum(self.theta_weights * self.beta_samples)
        nu_mean = np.sum(self.theta_weights * self.nu_samples)
        ts_prob = self._taskset_probability(self.theta_weights)

        self.history["vol_mean"].append(vol_mean)
        self.history["beta_mean"].append(beta_mean)
        self.history["nu_mean"].append(nu_mean)
        self.history["ts_prob"].append(ts_prob)

    # ------------------------------------------------------------------
    # Decision policy helpers
    # ------------------------------------------------------------------
    def _taskset_probability(self, theta_weights: np.ndarray) -> np.ndarray:
        ts_prob = np.zeros(K)
        for theta_idx in range(self.num_theta):
            counts = np.bincount(
                self.state_particles[theta_idx], minlength=K
            ) / self.num_state_particles
            ts_prob += theta_weights[theta_idx] * counts
        ts_prob /= np.sum(ts_prob)
        return ts_prob

    def _action_distribution(self, stimulus: int) -> np.ndarray:
        ts_prob = self._taskset_probability(self.theta_weights)
        action_probs = np.zeros(2)
        for action in range(2):
            mask = TASK_MAPPING[stimulus] == action
            action_probs[action] = ts_prob[mask].sum()
        if self.beta_softmax is None:
            greedy_action = np.argmax(action_probs)
            probs = np.zeros_like(action_probs)
            probs[greedy_action] = 1.0
            return probs
        # Findling et al.: softmax on LOG of beliefs, not raw probabilities
        # P(a) ∝ exp(β * log(belief_a)) = belief_a^β
        log_beliefs = np.log(np.clip(action_probs, 1e-10, 1.0))
        logits = log_beliefs * self.beta_softmax
        logits -= np.max(logits)  # Numerical stability
        probs = np.exp(logits)
        probs /= probs.sum()
        probs = probs * (1 - self.epsilon_softmax) + self.epsilon_softmax / len(probs)
        return probs

    def _sample_action(self, action_probs: np.ndarray) -> int:
        if np.isclose(action_probs.sum(), 0):
            return self.rng.integers(2)
        if np.count_nonzero(action_probs) == 1:
            return int(np.argmax(action_probs))
        return self.rng.choice(len(action_probs), p=action_probs)



### Step 3b: Weber-imprecision agent


In [8]:
class WeberImprecisionAgent:
    """Forward Weber-imprecision agent.
    
    Parameters (Paper vs Code mapping):
    ------------------------------------
    - beta_samples (internal): η/2 sampled from Beta(1,1), transformed to η = beta/2 + 0.5
      This is the FEEDBACK NOISE parameter (η > 0.5)
    - gamma_samples (internal): γ sampled from Dirichlet(1,...,1)
      These are the task-set COMBINATION PROBABILITIES
    - lambdaa: λ, the Weber scaling factor (FREE PARAMETER)
    - mu: μ, the Weber baseline noise (FREE PARAMETER). C++ default is 0.0!
    - beta_softmax: β, the softmax inverse temperature (FREE PARAMETER)
      NOTE: This is DIFFERENT from beta_samples (feedback noise η)!
    
    Weber noise formula: ε_t ~ U(0, μ + λ*d_t), applied as ε_t/2
    
    NOTE: Weber does NOT have nu_samples or tau_particles - those are VarVol only!
    """

    def __init__(
        self,
        num_theta: int = 200, # simulation uses 1000, fitting only 200
        num_state_particles: int = 200, # simulation uses 1000-2000, fitting only 200
        lambdaa: float = 0.4,      # C++ default for simulation 0.9, TO FIT 0.4
        mu: float = 0.1,           # C++ default for simulation 0.0, TO FIT 0.1
        beta_prior: Tuple[float, float] = (1.0, 1.0),
        gamma_prior: float = 1.0,
        ess_threshold: float = 0.5,
        beta_softmax: Optional[float] = 1.0,  # None = argmax policy in simulation
        epsilon_softmax: float = 0.0,
        seed: Optional[int] = None,
    ):
        self.num_theta = num_theta
        self.num_state_particles = num_state_particles
        self.lambdaa = lambdaa
        self.mu = mu
        self.beta_prior = beta_prior
        self.gamma_prior = gamma_prior
        self.ess_threshold = ess_threshold
        self.beta_softmax = beta_softmax
        self.epsilon_softmax = epsilon_softmax
        self.rng = np.random.default_rng(seed)

        self._init_particles()

    def _init_particles(self):
        self.beta_samples = self.rng.beta(
            self.beta_prior[0], self.beta_prior[1], size=self.num_theta
        )
        self.gamma_samples = self.rng.dirichlet(
            np.ones(K) * self.gamma_prior, size=self.num_theta
        )
        self.state_particles = self.rng.integers(
            K, size=(self.num_theta, self.num_state_particles)
        )
        self.log_theta_weights = np.zeros(self.num_theta)
        self.theta_weights = np.ones(self.num_theta) / self.num_theta
        self.pending_observation = None
        self.current_action = None
        self.current_stimulus = None

        self.history = {
            "epsilon": [],
            "beta_mean": [],
            "ts_prob": [],
        }

    def act(self, stimulus: int) -> Tuple[int, np.ndarray]:
        self._maybe_smc_update()
        action_probs = self._action_distribution(stimulus)
        action = self._sample_action(action_probs)
        self.current_action = action
        self.current_stimulus = stimulus
        return action, action_probs

    def observe(self, reward: int):
        self.pending_observation = (
            self.current_stimulus,
            self.current_action,
            reward,
        )

    def _maybe_smc_update(self):
        if self.pending_observation is None:
            return

        stimulus, action, reward = self.pending_observation
        prev_states = self.state_particles.copy()

        positive_states = compute_positive_states(TASK_MAPPING, stimulus, action, reward)
        beta_transformed = self.beta_samples / 2.0 + 0.5
        beta_matrix = beta_transformed[:, None]
        state_pos = positive_states[prev_states]
        likelihoods = state_pos * beta_matrix + (1 - state_pos) * (1 - beta_matrix)
        likelihoods = np.clip(likelihoods, 1e-9, None)

        # Compute ante (before update) belief
        ante_counts = (
            np.apply_along_axis(
                lambda row: np.bincount(row, minlength=K), 1, prev_states
            )
            / self.num_state_particles
        )
        
        sum_weights = likelihoods.sum(axis=1)
        weights_norm = likelihoods / sum_weights[:, None]
        
        # Compute post (after update) belief per theta
        weighted_post = np.zeros_like(ante_counts)
        for theta_idx in range(self.num_theta):
            for state_idx in range(self.num_state_particles):
                s = prev_states[theta_idx, state_idx]
                weighted_post[theta_idx, s] += weights_norm[theta_idx, state_idx]
        weighted_post = np.divide(
            weighted_post,
            weighted_post.sum(axis=1, keepdims=True) + 1e-12,
        )
        
        # === GLOBAL DISTANCE (matching C++) ===
        current_theta_weights = normalise_log_weights(self.log_theta_weights)
        global_ante = np.sum(current_theta_weights[:, None] * ante_counts, axis=0)
        
        updated_log_weights = self.log_theta_weights + np.log(sum_weights / self.num_state_particles)
        updated_theta_weights = normalise_log_weights(updated_log_weights)
        global_post = np.sum(updated_theta_weights[:, None] * weighted_post, axis=0)
        
        global_distance = np.sum(np.abs(global_ante - global_post))
        epsilon = (self.lambdaa * global_distance + self.mu) / 2.0
        epsilon = np.clip(epsilon, 0.0, 0.5)
        
        self.history["epsilon"].append(epsilon)

        self.log_theta_weights += np.log(sum_weights / self.num_state_particles)

        new_states = np.empty_like(prev_states)
        for theta_idx in range(self.num_theta):
            ancestor_idx = stratified_resample(weights_norm[theta_idx], self.rng)
            candidates = prev_states[theta_idx, ancestor_idx]
            keep_mask = self.rng.random(self.num_state_particles) > epsilon  # Single epsilon
            for s_idx in range(self.num_state_particles):
                if keep_mask[s_idx]:
                    new_states[theta_idx, s_idx] = candidates[s_idx]
                else:
                    if K == 2:
                        new_states[theta_idx, s_idx] = 1 - candidates[s_idx]
                    else:
                        new_states[theta_idx, s_idx] = sample_new_state_excluding(
                            candidates[s_idx], self.gamma_samples[theta_idx], self.rng
                        )
        self.state_particles = new_states

        self.pending_observation = None
        self.theta_weights = normalise_log_weights(self.log_theta_weights)
        self._maybe_rejuvenate()
        self._record_history()

    def _maybe_rejuvenate(self):
        ess = 1.0 / np.sum(self.theta_weights ** 2)
        if ess >= self.ess_threshold * self.num_theta:
            return

        beta_mu = np.sum(self.theta_weights * self.beta_samples)
        beta_var = np.sum(self.theta_weights * (self.beta_samples - beta_mu) ** 2)
        beta_var = max(beta_var, 1e-6)
        beta_alpha = max(((1 - beta_mu) / beta_var - 1 / beta_mu) * beta_mu ** 2, 1.0)
        beta_beta = max(beta_alpha * (1 / beta_mu - 1), 1.0)

        dirichlet_means = np.sum(
            self.theta_weights[:, None] * self.gamma_samples, axis=0
        )
        dirichlet_vars = np.sum(
            self.theta_weights[:, None] * (self.gamma_samples ** 2), axis=0
        ) - dirichlet_means ** 2
        dirichlet_vars = np.clip(dirichlet_vars, 1e-6, None)
        dirichlet_precision = (
            np.sum(dirichlet_means - dirichlet_means ** 2) / np.sum(dirichlet_vars)
        ) - 1
        dirichlet_precision = max(dirichlet_precision, 1.0)
        dirichlet_params = np.maximum(
            dirichlet_means * dirichlet_precision, 1.0
        )

        self.beta_samples = self.rng.beta(
            beta_alpha, beta_beta, size=self.num_theta
        )
        # Weber does NOT have nu_samples - only beta and gamma!
        self.gamma_samples = self.rng.dirichlet(
            dirichlet_params, size=self.num_theta
        )
        self.log_theta_weights[:] = 0.0
        self.theta_weights = np.ones(self.num_theta) / self.num_theta

    def _record_history(self):
        beta_mean = np.sum(self.theta_weights * self.beta_samples)
        ts_prob = self._taskset_probability()
        self.history["beta_mean"].append(beta_mean)
        self.history["ts_prob"].append(ts_prob)

    def _taskset_probability(self) -> np.ndarray:
        ts_prob = np.zeros(K)
        for theta_idx in range(self.num_theta):
            counts = np.bincount(
                self.state_particles[theta_idx], minlength=K
            ) / self.num_state_particles
            ts_prob += self.theta_weights[theta_idx] * counts
        ts_prob /= np.sum(ts_prob)
        return ts_prob

    def _action_distribution(self, stimulus: int) -> np.ndarray:
        ts_prob = self._taskset_probability()
        action_probs = np.zeros(2)
        for action in range(2):
            mask = TASK_MAPPING[stimulus] == action
            action_probs[action] = ts_prob[mask].sum()
        if self.beta_softmax is None:
            greedy_action = np.argmax(action_probs)
            probs = np.zeros_like(action_probs)
            probs[greedy_action] = 1.0
            return probs
        # Findling et al.: softmax on LOG of beliefs, not raw probabilities
        # P(a) ∝ exp(β * log(belief_a)) = belief_a^β
        log_beliefs = np.log(np.clip(action_probs, 1e-10, 1.0))
        logits = log_beliefs * self.beta_softmax
        logits -= np.max(logits)  # Numerical stability
        probs = np.exp(logits)
        probs /= probs.sum()
        probs = probs * (1 - self.epsilon_softmax) + self.epsilon_softmax / len(probs)
        return probs

    def _sample_action(self, action_probs: np.ndarray) -> int:
        if np.isclose(action_probs.sum(), 0):
            return self.rng.integers(2)
        if np.count_nonzero(action_probs) == 1:
            return int(np.argmax(action_probs))
        return self.rng.choice(len(action_probs), p=action_probs)



### Step 4: Parameter Fitting

Settings for fitting:

We use Differential Evolution instead of Bayesian Optimization like in Findling - for speed. 

Findling et al. (2021) used:
   - EXACT models: 1000 theta × 1000 state = 1M particles 
   - FORWARD models: 200 theta × 200 state = 40K particles 

 For a seminar project 100×100 is used for efficiency.



In [9]:

# ============================================================
# UTILITY FUNCTIONS
# ============================================================

# FITTING SETTINGS - DIFFERENTIAL EVOLUTION

# Particle filter settings
FITTING_N_THETA = 100
FITTING_N_STATE = 100

# PF runs for likelihood estimation (marginalization)
N_PF_RUNS = 10  # Average over this many PF realizations per evaluation

# Differential Evolution settings (tuned from test: 9 min per participant)
DE_MAXITER = 15          # Generations
DE_POPSIZE = 5           # Population size multiplier (actual pop = popsize * n_params)
DE_TOL = 0.1             # Convergence tolerance
DE_SEED = 42             # Reproducibility

# Parallelization
N_WORKERS = max(1, os.cpu_count() - 2)  # Leave 2 cores free

print("="*70)
print(" FITTING SETTINGS")
print("="*70)
print(f"Particles: {FITTING_N_THETA} x {FITTING_N_STATE} = {FITTING_N_THETA * FITTING_N_STATE:,}")
print(f"PF runs per evaluation: {N_PF_RUNS}")
print(f"DE settings: maxiter={DE_MAXITER}, popsize={DE_POPSIZE}, tol={DE_TOL}")
print(f"Parallel workers: {N_WORKERS} (of {os.cpu_count()} cores)")

# ============================================================
# Reconstruct task from human data
# ============================================================
def reconstruct_task_from_human_data(participant_df: pd.DataFrame) -> Tuple[VolatileBanditTask, np.ndarray]:
    """
    Reconstruct a VolatileBanditTask from human behavioral data.
    
    KEY FIELDS USED:
    ----------------
    - hidden_state: Which arm was correct on each trial (0 or 1)
      → This IS the latent state AND the correct action (identity mapping)
      → Switches are encoded here: [0,0,0,1,1,1,...] means switch at trial 3
    
    - false_feedback: Whether feedback was flipped (trap) on each trial
      → True means human saw opposite of what they should have
    
    - outcome: The actual reward the human received ("win" or "loss")
      → This is what we feed to the agent during fitting!
    
    FIELDS NOT USED IN FITTING (just for bookkeeping):
    --------------------------------------------------
    - stimuli: Always 0 (2-armed bandit has 1 stimulus type)
    - switch_prob: Average switch rate (not used - we have exact switches)
    - beta: Average trap rate (not used - we have exact traps)
    
    Returns:
        task: VolatileBanditTask with exact trial-by-trial structure
        human_rewards: The actual rewards the human received (0 or 1)
    """
    n_trials = len(participant_df)
    
    # Extract exact trial-by-trial data (not averages!)
    correct_actions = participant_df['hidden_state'].values.astype(int)
    traps = participant_df['false_feedback'].values.astype(bool)
    
    # The ACTUAL reward the human saw (what we feed to the agent!)
    human_rewards = (participant_df['outcome'] == 'win').values.astype(int)
    
    # These are for bookkeeping only - not used in fitting
    switches = np.sum(np.diff(correct_actions) != 0)
    switch_rate = switches / (n_trials - 1) if n_trials > 1 else 0.0
    switch_prob = np.full(n_trials, switch_rate) #unused
    trap_rate = np.mean(traps)
    beta = 1.0 - trap_rate # unused
    
    # Create task with trial-by-trial data
    stimuli = np.zeros(n_trials, dtype=int)
    latent_states = correct_actions.copy()
    
    task = VolatileBanditTask(
        stimuli=stimuli,
        latent_states=latent_states,
        switch_prob=switch_prob, # not used in determinstic human task
        traps=traps,
        correct_actions=correct_actions,
        beta=beta,
    )
    
    return task, human_rewards

# ============================================================
# Negative Log-Likelihood functions (to minimize)
# ============================================================
# Following Findling et al. (2021):
#   - VarVol: 1 free parameter (β = inverse temperature)
#   - Weber:  3 free parameters (β, μ, λ)
#
# We average over multiple PF realizations to get stable likelihood estimates
# (equivalent to Findling's numberOfBetaSamples marginalization)

def compute_neg_log_likelihood_varvol(params, task, human_choices, human_rewards, n_runs=N_PF_RUNS*2):
    """
    Compute negative log-likelihood for VarVol model.
    Averages over n_runs particle filter realizations for stability.
    
    params: [beta_softmax]
    """
    beta_softmax = params[0]
    beta_softmax = max(0.01, beta_softmax)
    
    log_liks = []
    for run_seed in range(n_runs):
        try:
            agent = ForwardVaryingVolatilityAgent(
                num_theta=FITTING_N_THETA,
                num_state_particles=FITTING_N_STATE,
                beta_softmax=beta_softmax,
                seed=42 + run_seed  # Different seed each run
            )
            
            log_lik = 0.0
            for t in range(len(human_choices)):
                stimulus = task.stimuli[t]
                _, action_probs = agent.act(stimulus)
                
                human_choice = human_choices[t]
                prob_human_choice = np.clip(action_probs[human_choice], 1e-10, 1.0)
                log_lik += np.log(prob_human_choice)
                
                agent.current_action = human_choice
                agent.current_stimulus = stimulus
                agent.observe(human_rewards[t])
            
            log_liks.append(log_lik)
        except Exception as e:
            log_liks.append(-1e8)  # Large negative on error
    
    # Average over PF realizations (marginalization)
    mean_log_lik = np.mean(log_liks)
    return -mean_log_lik  # Negative because we minimize


def compute_neg_log_likelihood_weber(params, task, human_choices, human_rewards, n_runs=N_PF_RUNS):
    """
    Compute negative log-likelihood for Weber model.
    Averages over n_runs particle filter realizations for stability.
    
    params: [beta_softmax, lambdaa, mu]
    """
    beta_softmax, lambdaa, mu = params
    beta_softmax = max(0.01, beta_softmax)
    lambdaa = np.clip(lambdaa, 0.01, 2.0)
    mu = np.clip(mu, 0.0, 1.0)
    
    log_liks = []
    for run_seed in range(n_runs):
        try:
            agent = WeberImprecisionAgent(
                num_theta=FITTING_N_THETA,
                num_state_particles=FITTING_N_STATE,
                beta_softmax=beta_softmax,
                lambdaa=lambdaa,
                mu=mu,
                seed=42 + run_seed  # Different seed each run
            )
            
            log_lik = 0.0
            for t in range(len(human_choices)):
                stimulus = task.stimuli[t]
                _, action_probs = agent.act(stimulus)
                
                human_choice = human_choices[t]
                prob_human_choice = np.clip(action_probs[human_choice], 1e-10, 1.0)
                log_lik += np.log(prob_human_choice)
                
                agent.current_action = human_choice
                agent.current_stimulus = stimulus
                agent.observe(human_rewards[t])
            
            log_liks.append(log_lik)
        except Exception as e:
            log_liks.append(-1e8)
    
    mean_log_lik = np.mean(log_liks)
    return -mean_log_lik # Negative because we minimize


# ============================================================
# FITTING FUNCTIONS: DIFFERENTIAL EVOLUTION
# ============================================================
# DE is a global optimizer - it does NOT need multiple restarts like L-BFGS-B.
# It maintains a population of candidate solutions and evolves them,
# inherently exploring the full parameter space.

def fit_varvol_to_participant(task, human_choices, human_rewards):
    """
    Fit VarVol model using Differential Evolution.
    VarVol has 1 free parameter: beta (softmax inverse temperature)
    """
    bounds = [(0.1, 15.0)]  # beta_softmax
    
    result = differential_evolution(
        compute_neg_log_likelihood_varvol,
        bounds,
        args=(task, human_choices, human_rewards),
        seed=DE_SEED,
        maxiter=DE_MAXITER,
        popsize=DE_POPSIZE,
        tol=DE_TOL,
        workers=1,  # Do not parallelize internally (we parallelize across participants)
        updating='deferred',
        disp=False
    )
    
    return {
        'beta_softmax': result.x[0],
        'neg_log_lik': result.fun,
        'log_lik': -result.fun,
        'success': result.success,
        'n_params': 1,
        'n_evals': result.nfev
    }


def fit_weber_to_participant(task, human_choices, human_rewards):
    """
    Fit Weber model using Differential Evolution.
    Weber has 3 free parameters: beta (softmax), lambda (Weber scaling), mu (baseline noise)
    
    DE will properly explore the lambda and mu parameter space,
    unlike L-BFGS-B which got stuck at starting values.
    """
    bounds = [
        (0.1, 15.0),   # beta_softmax
        (0.0, 2.0),    # lambdaa (Weber scaling)
        (0.0, 1.0)     # mu (baseline noise)
    ]
    
    result = differential_evolution(
        compute_neg_log_likelihood_weber,
        bounds,
        args=(task, human_choices, human_rewards),
        seed=DE_SEED,
        maxiter=DE_MAXITER,
        popsize=DE_POPSIZE,
        tol=DE_TOL,
        workers=1,  # Do not parallelize internally
        updating='deferred',
        disp=False
    )
    
    return {
        'beta_softmax': result.x[0],
        'lambdaa': result.x[1],
        'mu': result.x[2],
        'neg_log_lik': result.fun,
        'log_lik': -result.fun,
        'success': result.success,
        'n_params': 3,
        'n_evals': result.nfev
    }


 FITTING SETTINGS
Particles: 100 x 100 = 10,000
PF runs per evaluation: 10
DE settings: maxiter=15, popsize=5, tol=0.1
Parallel workers: 20 (of 22 cores)


In [ ]:
# ============================================================
# PARALLEL FITTING WITH DIFFERENTIAL EVOLUTION
# ============================================================

print("="*70)
print(" MLE PARAMETER FITTING: Differential Evolution")
print("="*70)

DATA_DIR = Path('2.fitting_data/extended_csvs')
CSV_FILES = sorted(DATA_DIR.glob('*.csv'))

print(f"\nFound {len(CSV_FILES)} participants")
print(f"Estimated time: ~{len(CSV_FILES) * 9 / N_WORKERS:.0f} minutes with {N_WORKERS} workers")


def fit_one_participant(csv_file):
    """Fit both models to one participant using Differential Evolution."""
    participant_id = csv_file.stem
    try:
        df = pd.read_csv(csv_file)
        required_cols = ['hidden_state', 'false_feedback', 'outcome', 'userChoice']
        if not all(col in df.columns for col in required_cols):
            return None

        task, human_rewards = reconstruct_task_from_human_data(df)
        human_choices = df['userChoice'].values.astype(int)
        n_trials = len(human_choices)

        # Fit VarVol with DE
        varvol_fit = fit_varvol_to_participant(task, human_choices, human_rewards)
        
        # Fit Weber with DE
        weber_fit = fit_weber_to_participant(task, human_choices, human_rewards)

        # Compute BIC: k * ln(n) + 2 * NLL
        varvol_bic = varvol_fit['n_params'] * np.log(n_trials) + 2 * varvol_fit['neg_log_lik']
        weber_bic = weber_fit['n_params'] * np.log(n_trials) + 2 * weber_fit['neg_log_lik']

        return {
            'participant_id': participant_id,
            'n_trials': n_trials,
            # VarVol results
            'varvol_beta_softmax': varvol_fit['beta_softmax'],
            'varvol_log_lik': varvol_fit['log_lik'],
            'varvol_bic': varvol_bic,
            'varvol_n_params': varvol_fit['n_params'],
            'varvol_success': varvol_fit['success'],
            'varvol_n_evals': varvol_fit['n_evals'],
            # Weber results
            'weber_beta_softmax': weber_fit['beta_softmax'],
            'weber_lambdaa': weber_fit['lambdaa'],
            'weber_mu': weber_fit['mu'],
            'weber_log_lik': weber_fit['log_lik'],
            'weber_bic': weber_bic,
            'weber_n_params': weber_fit['n_params'],
            'weber_success': weber_fit['success'],
            'weber_n_evals': weber_fit['n_evals'],
            # Comparison
            'delta_bic': varvol_bic - weber_bic,
            'better_model': 'VarVol' if varvol_bic < weber_bic else 'Weber',
        }
    
    except Exception as e:
        print(f"Error {participant_id}: {e}")
        return None


# ============================================================
# RUN PARALLEL FITTING
# ============================================================
print(f"\nStarting parallel fitting...")
start_time = time.time()

results = Parallel(n_jobs=N_WORKERS, verbose=10)(
    delayed(fit_one_participant)(csv_file) for csv_file in CSV_FILES
)

elapsed = time.time() - start_time

# Filter failures
results = [r for r in results if r is not None]
fitting_df = pd.DataFrame(results)

print("\n" + "="*70)
print(" FITTING COMPLETE")
print("="*70)
print(f"Time: {elapsed/60:.1f} minutes ({elapsed/3600:.2f} hours)")
print(f"Avg per participant: {elapsed/len(fitting_df):.1f} seconds")
print(f"Successfully fitted: {len(fitting_df)}/{len(CSV_FILES)}")

# Save results
output_path = Path('2.fitting_data/mle_fitting_results_DE.csv')
fitting_df.to_csv(output_path, index=False)
print(f"\nSaved: {output_path}")

 MLE PARAMETER FITTING: Differential Evolution

Found 45 participants
Estimated time: ~20 minutes with 20 workers

Starting parallel fitting...


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


In [ ]:
print("Fitted parameters:")
fitting_df = pd.read_csv('2.fitting_data/mle_fitting_results_DE.csv')
print(fitting_df[['participant_id', 'varvol_beta_softmax', 'weber_beta_softmax', 
                   'weber_lambdaa', 'weber_mu', 'varvol_log_lik', 'weber_log_lik']])

Fitted parameters:
                          participant_id  varvol_beta_softmax  \
0   25da8b12-9a43-40cd-84f2-5b152a5f0f7f             0.250018   
1                  adaptive_lab_07ee303f             1.629521   
2                  adaptive_lab_08d9887c             2.079174   
3                  adaptive_lab_09b45851             0.415455   
4                  adaptive_lab_1671e99c             1.372539   
5                  adaptive_lab_1ea56ad2             3.367573   
6                  adaptive_lab_22a6875c             0.610808   
7                  adaptive_lab_2eade079             2.501466   
8                  adaptive_lab_3162592a             1.130284   
9                  adaptive_lab_390371f9             0.010000   
10                 adaptive_lab_42003269             1.195727   
11                 adaptive_lab_4b7a6ad7             1.203803   
12                 adaptive_lab_52ae4c1c             0.010000   
13                 adaptive_lab_534d2332             0.383759   
14    

### Step 5: Summary and Visualisations

In [ ]:
# ============================================================
# RESULTS SUMMARY
# ============================================================

print("="*70)
print(" MLE FITTING RESULTS SUMMARY")
print("="*70)

n_participants = len(fitting_df)
n_varvol_wins = (fitting_df['better_model'] == 'VarVol').sum()
n_weber_wins = (fitting_df['better_model'] == 'Weber').sum()

print(f"\nParticipants fitted: {n_participants}")
print(f"VarVol wins (lower BIC): {n_varvol_wins} ({100*n_varvol_wins/n_participants:.1f}%)")
print(f"Weber wins (lower BIC):  {n_weber_wins} ({100*n_weber_wins/n_participants:.1f}%)")

print(f"\n{'Metric':<25} {'VarVol':<15} {'Weber':<15}")
print("-"*55)
print(f"{'Mean Log-Likelihood':<25} {fitting_df['varvol_log_lik'].mean():<15.2f} {fitting_df['weber_log_lik'].mean():<15.2f}")
print(f"{'Mean BIC':<25} {fitting_df['varvol_bic'].mean():<15.2f} {fitting_df['weber_bic'].mean():<15.2f}")
print(f"{'Sum BIC':<25} {fitting_df['varvol_bic'].sum():<15.2f} {fitting_df['weber_bic'].sum():<15.2f}")

delta_bic_mean = fitting_df['delta_bic'].mean()
delta_bic_sum = fitting_df['delta_bic'].sum()

print(f"\n{'Mean ΔBIC (VV-Weber)':<25} {delta_bic_mean:<15.2f}")
print(f"{'Sum ΔBIC (VV-Weber)':<25} {delta_bic_sum:<15.2f}")

if delta_bic_sum < 0:
    print(f"\n--> Overall: VarVol is favored (sum ΔBIC = {delta_bic_sum:.1f})")
else:
    print(f"\n--> Overall: Weber is favored (sum ΔBIC = {delta_bic_sum:.1f})")

# Fitted parameter distributions
print("\n" + "="*70)
print(" FITTED PARAMETER DISTRIBUTIONS")
print("="*70)

print(f"\nVarVol (1 free param):")
print(f"  beta: mean={fitting_df['varvol_beta_softmax'].mean():.2f}, "
      f"std={fitting_df['varvol_beta_softmax'].std():.2f}, "
      f"range=[{fitting_df['varvol_beta_softmax'].min():.2f}, {fitting_df['varvol_beta_softmax'].max():.2f}]")

print(f"\nWeber (3 free params):")
print(f"  beta:   mean={fitting_df['weber_beta_softmax'].mean():.2f}, "
      f"std={fitting_df['weber_beta_softmax'].std():.2f}, "
      f"range=[{fitting_df['weber_beta_softmax'].min():.2f}, {fitting_df['weber_beta_softmax'].max():.2f}]")
print(f"  lambda: mean={fitting_df['weber_lambdaa'].mean():.2f}, "
      f"std={fitting_df['weber_lambdaa'].std():.2f}, "
      f"range=[{fitting_df['weber_lambdaa'].min():.2f}, {fitting_df['weber_lambdaa'].max():.2f}]")
print(f"  mu:     mean={fitting_df['weber_mu'].mean():.3f}, "
      f"std={fitting_df['weber_mu'].std():.3f}, "
      f"range=[{fitting_df['weber_mu'].min():.3f}, {fitting_df['weber_mu'].max():.3f}]")

print("\n" + "="*70)
print(" FITTED PARAMETERS PER PARTICIPANT")
print("="*70)
print(fitting_df[['participant_id', 'varvol_beta_softmax', 'weber_beta_softmax', 
                  'weber_lambdaa', 'weber_mu', 'delta_bic', 'better_model']])

In [ ]:
# ============================================================
# SANITY CHECK: Did DE actually explore lambda and mu?
# ============================================================

print("="*70)
print(" SANITY CHECK: Weber Parameter Exploration")
print("="*70)

print(f"\nWeber lambda distribution:")
print(f"   Mean:  {fitting_df['weber_lambdaa'].mean():.3f}")
print(f"   Std:   {fitting_df['weber_lambdaa'].std():.3f}")
print(f"   Min:   {fitting_df['weber_lambdaa'].min():.3f}")
print(f"   Max:   {fitting_df['weber_lambdaa'].max():.3f}")

print(f"\nWeber mu distribution:")
print(f"   Mean:  {fitting_df['weber_mu'].mean():.3f}")
print(f"   Std:   {fitting_df['weber_mu'].std():.3f}")
print(f"   Min:   {fitting_df['weber_mu'].min():.3f}")
print(f"   Max:   {fitting_df['weber_mu'].max():.3f}")

# Check if parameters are stuck (the problem we fixed!)
lambda_stuck = fitting_df['weber_lambdaa'].std() < 0.05
mu_stuck = fitting_df['weber_mu'].std() < 0.02

if lambda_stuck or mu_stuck:
    print("\n[WARNING] Parameters may still be stuck!")
    if lambda_stuck:
        print(f"   lambda has very low variance (std={fitting_df['weber_lambdaa'].std():.3f})")
    if mu_stuck:
        print(f"   mu has very low variance (std={fitting_df['weber_mu'].std():.3f})")
else:
    print("\n[OK] Parameters show healthy variation - DE is working!")

# Show unique values to verify exploration
n_unique_lambda = fitting_df['weber_lambdaa'].nunique()
n_unique_mu = fitting_df['weber_mu'].nunique()

print(f"\nUnique lambda values: {n_unique_lambda} (should be close to {len(fitting_df)})")
print(f"Unique mu values: {n_unique_mu} (should be close to {len(fitting_df)})")

# Compare to old stuck values
print("\n" + "-"*70)
print("Comparison to L-BFGS-B stuck values:")
print("-"*70)
print(f"Old stuck lambda=0.9: {(fitting_df['weber_lambdaa'] == 0.9).sum()} participants")
print(f"Old stuck mu=0.0:     {(fitting_df['weber_mu'] == 0.0).sum()} participants")

if (fitting_df['weber_lambdaa'] == 0.9).sum() > len(fitting_df) * 0.5:
    print("\n[WARNING] Many participants still have lambda=0.9 - check DE settings!")
else:
    print("\n[OK] DE successfully moved away from stuck values!")

In [ ]:
### VISUALIZATION: BIC and Log-Likelihood per participant

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Model Comparison: BIC and Accuracy', fontsize=14, fontweight='bold')

# Panel 1: BIC per participant
ax = axes[0]
x = np.arange(len(fitting_df))
width = 0.35
ax.bar(x - width/2, fitting_df['varvol_bic'], width, label='VarVol', color="#6eaaa2", alpha=0.8)
ax.bar(x + width/2, fitting_df['weber_bic'], width, label='Weber', color="#d27763", alpha=0.8)
ax.set_xlabel('Participant')
ax.set_ylabel('BIC (lower = better)')
ax.set_title('BIC per Participant')
ax.legend()
ax.set_xticks(x[::5])

# Panel 2: Log-likelihood per participant
ax = axes[1]
ax.bar(x - width/2, fitting_df['varvol_log_lik'], width, label='VarVol', color="#6eaaa2", alpha=0.8)
ax.bar(x + width/2, fitting_df['weber_log_lik'], width, label='Weber', color="#d27763", alpha=0.8)
ax.set_xlabel('Participant')
ax.set_ylabel('Log-Likelihood (higher = better)')
ax.set_title('Log-Likelihood per Participant')
ax.legend()
ax.set_xticks(x[::5])

plt.tight_layout()
plt.show()

# Panel 3: Pie Count wins
n_varvol_wins = (fitting_df['delta_bic'] < 0).sum()
n_weber_wins = (fitting_df['delta_bic'] >= 0).sum()

fig, ax = plt.subplots(figsize=(5, 5))
colors = ["#6eaaa2", "#d27763"]
explode = (0.05, 0.05)
ax.pie([n_varvol_wins, n_weber_wins],
       labels=[f'VarVol\n({n_varvol_wins})', f'Weber\n({n_weber_wins})'],
       autopct='%1.1f%%', colors=colors, explode=explode,
       shadow=True, startangle=90)
ax.set_title('Model Wins (BIC)')
plt.tight_layout()
plt.show()

In [ ]:
# Parameter distributions

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Fitted Parameter Distributions', fontsize=14, fontweight='bold')

# β distribution
ax = axes[0]
ax.hist(fitting_df['varvol_beta_softmax'], bins=15, alpha=0.7, label='VarVol β', color="#6eaaa2")
ax.hist(fitting_df['weber_beta_softmax'], bins=15, alpha=0.7, label='Weber β', color="#d27763")
ax.set_xlabel('β (inverse temperature)')
ax.set_ylabel('Count')
ax.set_title('Softmax β Distribution')
ax.legend()

# λ distribution
ax = axes[1]
ax.hist(fitting_df['weber_lambdaa'], bins=15, color="#d27763", alpha=0.7, edgecolor='black')
ax.axvline(fitting_df['weber_lambdaa'].mean(), color='black', linestyle='--', linewidth=2,
           label=f'Mean: {fitting_df['weber_lambdaa'].mean():.2f}')
ax.set_xlabel('λ (Weber scaling)')
ax.set_ylabel('Count')
ax.set_title('Weber λ Distribution')
ax.legend()

# μ distribution
ax = axes[2]
ax.hist(fitting_df['weber_mu'], bins=15, color="#d27763", alpha=0.7, edgecolor='black')
ax.axvline(fitting_df['weber_mu'].mean(), color='black', linestyle='--', linewidth=2,
           label=f'Mean: {fitting_df['weber_mu'].mean():.3f}')
ax.set_xlabel('μ (baseline noise)')
ax.set_ylabel('Count')
ax.set_title('Weber μ Distribution')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Choice match rates (model vs human) - seed variance analysis

# Small seed test: average over 5 seeds for each participant
N_SEEDS = 5
choice_match_results = []

for i, row in fitting_df.iterrows():
    participant_id = row['participant_id']
    csv_file = DATA_DIR / f"{participant_id}.csv"
    if not csv_file.exists():
        continue
    df = pd.read_csv(csv_file)
    task, human_rewards = reconstruct_task_from_human_data(df)
    human_choices = df['userChoice'].values.astype(int)
    n_trials = len(human_choices)

    varvol_matches = []
    weber_matches = []
    for seed in range(N_SEEDS):
        varvol_agent = ForwardVaryingVolatilityAgent(
            num_theta=FITTING_N_THETA,
            num_state_particles=FITTING_N_STATE,
            beta_softmax=row['varvol_beta_softmax'],
            seed=seed
        )
        weber_agent = WeberImprecisionAgent(
            num_theta=FITTING_N_THETA,
            num_state_particles=FITTING_N_STATE,
            beta_softmax=row['weber_beta_softmax'],
            lambdaa=row['weber_lambdaa'],
            mu=row['weber_mu'],
            seed=seed
        )
        varvol_choices = []
        weber_choices = []
        for t in range(n_trials):
            stimulus = task.stimuli[t]
            vv_action, _ = varvol_agent.act(stimulus)
            varvol_choices.append(vv_action)
            varvol_agent.current_action = human_choices[t]
            varvol_agent.current_stimulus = stimulus
            varvol_agent.observe(human_rewards[t])
            wb_action, _ = weber_agent.act(stimulus)
            weber_choices.append(wb_action)
            weber_agent.current_action = human_choices[t]
            weber_agent.current_stimulus = stimulus
            weber_agent.observe(human_rewards[t])
        varvol_matches.append(np.mean(np.array(varvol_choices) == human_choices))
        weber_matches.append(np.mean(np.array(weber_choices) == human_choices))

    choice_match_results.append({
        'participant_id': participant_id,
        'varvol_match_mean': np.mean(varvol_matches),
        'varvol_match_std': np.std(varvol_matches),
        'weber_match_mean': np.mean(weber_matches),
        'weber_match_std': np.std(weber_matches)
    })


match_df = pd.DataFrame(choice_match_results)

print("Seed variance summary (averaged across all participants):")
print(f"VarVol: mean match={match_df['varvol_match_mean'].mean():.3f}, mean seed std={match_df['varvol_match_std'].mean():.3f}")
print(f"Weber:  mean match={match_df['weber_match_mean'].mean():.3f}, mean seed std={match_df['weber_match_std'].mean():.3f}")

# Boxplot of choice match rates across participants
data_to_plot = [match_df['varvol_match_mean'], match_df['weber_match_mean']]
labels = ['VarVol', 'Weber']
colors = ["#6eaaa2", "#d27763"]

fig, ax = plt.subplots(figsize=(6, 5))
bp = ax.boxplot(data_to_plot, patch_artist=True, labels=labels)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_ylabel('Choice Match Rate')
ax.set_title('Model-Human Choice Match Rate Across Participants (Seed-Averaged)')
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()


# # Add condition info
# condition_info = {}
# for csv_file in sorted(DATA_DIR.glob('*.csv')):
#     try:
#         participant_id = csv_file.stem
#         df = pd.read_csv(csv_file)
#         instruction = df['instructionCondition'].iloc[0]
#         noise = df['condition'].mode()[0] if 'condition' in df.columns else 'unknown'
#         condition_info[participant_id] = {
#             'instruction': instruction,
#             'noise': noise
#         }
#     except Exception as e:
#         pass

# match_df['instruction'] = match_df['participant_id'].map(lambda x: condition_info.get(x, {}).get('instruction', 'unknown'))
# match_df['noise'] = match_df['participant_id'].map(lambda x: condition_info.get(x, {}).get('noise', 'unknown'))

# conditions = sorted(match_df[['instruction', 'noise']].drop_duplicates().values.tolist())

# fig, ax = plt.subplots(figsize=(8, 6))
# labels = []
# vv_data = []
# wb_data = []
# for c in conditions:
#     if c[0] != 'unknown':
#         subset = match_df[(match_df['instruction']==c[0]) & (match_df['noise']==c[1])]
#         vv_data.append(subset['varvol_match_human'].values)
#         wb_data.append(subset['weber_match_human'].values)
#         labels.append(f"{c[0]}-{c[1]}")

# bp1 = ax.boxplot(vv_data, positions=np.arange(len(labels))-0.2, widths=0.3, patch_artist=True, labels=labels)
# bp2 = ax.boxplot(wb_data, positions=np.arange(len(labels))+0.2, widths=0.3, patch_artist=True)
# for patch in bp1['boxes']:
#     patch.set_facecolor("#6eaaa2")
#     patch.set_alpha(0.7)
# for patch in bp2['boxes']:
#     patch.set_facecolor("#d27763")
#     patch.set_alpha(0.7)
# ax.set_ylabel('Choice Match Rate')
# ax.set_title('Choice Match Rate by Condition')
# ax.set_xticks(np.arange(len(labels)))
# ax.set_xticklabels(labels, rotation=30)
# ax.legend([bp1["boxes"][0], bp2["boxes"][0]], ['VarVol', 'Weber'])
# plt.tight_layout()
# plt.show()

NameError: name 'fitting_df' is not defined

In [ ]:
# human accuracy per group 

block_accuracy_results = []

for csv_file in sorted(DATA_DIR.glob('*.csv')):
    df = pd.read_csv(csv_file)
    participant_id = csv_file.stem
    instruction = df['instructionCondition'].iloc[0]
    blocks = df['block'].unique()
    for block in blocks:
        block_df = df[df['block'] == block]
        noise = block_df['condition'].iloc[0] if 'condition' in block_df.columns else 'unknown'
        human_choices = block_df['userChoice'].values.astype(int)
        correct_actions = block_df['hidden_state'].values.astype(int)
        human_accuracy = np.mean(human_choices == correct_actions)
        block_accuracy_results.append({
            'participant_id': participant_id,
            'instruction': instruction,
            'noise': noise,
            'block': block,
            'human_accuracy': human_accuracy
        })

block_acc_df = pd.DataFrame(block_accuracy_results)

# Group by instruction and noise
groups = sorted(block_acc_df[['instruction', 'noise']].drop_duplicates().values.tolist())
labels = []
data = []
for g in groups:
    if g[0] != 'unknown' and g[1] != 'unknown':
        subset = block_acc_df[(block_acc_df['instruction']==g[0]) & (block_acc_df['noise']==g[1])]
        data.append(subset['human_accuracy'].values)
        labels.append(f"{g[0]}-{g[1]}")
        print('check the block subsets. should be 2 blocks per participant so 90 in total?')
        print(f"{g[0]}-{g[1]}: {len(subset)} blocks")

fig, ax = plt.subplots(figsize=(8, 6))
bp = ax.boxplot(data, patch_artist=True, labels=labels)
for patch in bp['boxes']:
    patch.set_facecolor("#cccccc")
    patch.set_alpha(0.7)
ax.set_ylabel('Human Accuracy')
ax.set_title('Human Accuracy by Instruction and Noise (Block-Level)')
ax.set_xticks(np.arange(1, len(labels)+1))
ax.set_xticklabels(labels, rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
import os
print(os.cpu_count())

22
